# Importing Dependencies

In [2]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# Load Environment

In [11]:
env_name = "CartPole-v1"
env = gym.make(env_name, render_mode="human")

In [12]:
episodes = 5

for episode in range(1, episodes + 1):
    state, info = env.reset()
    done = False
    score = 0

    while not done:
        action = env.action_space.sample()
        next_state, reward, terminated, truncated, info = env.step(action)

        done = terminated or truncated
        score += reward
        state = next_state

    print(f"Episode {episode} score: {score}")

env.close()

Episode 1 score: 42.0
Episode 2 score: 15.0
Episode 3 score: 42.0
Episode 4 score: 13.0
Episode 5 score: 16.0


## Understanding the Environment

In [17]:
env.action_space

Discrete(2)

In [18]:
env.observation_space

Box([-4.8               -inf -0.41887903        -inf], [4.8               inf 0.41887903        inf], (4,), float32)

In [19]:
env.action_space.sample()

np.int64(1)

In [20]:
env.observation_space.sample()

array([-0.26743934, -0.3295444 , -0.19192973,  0.5796551 ], dtype=float32)

# Train RL Model

In [23]:
log_path = os.path.join("training", "logs")
os.makedirs(log_path, exist_ok=True)

In [25]:
env = gym.make(env_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [27]:
model.learn(total_timesteps=100000)

Logging to training\logs\PPO_1
-----------------------------
| time/              |      |
|    fps             | 1565 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1046        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008467808 |
|    clip_fraction        | 0.0859      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.000752    |
|    learning_rate        | 0.0003      |
|    loss                 | 7.87        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0131     |
|    value_loss           | 54.7        |
-----------------------------------------
---

# Save and Reload Model

In [28]:
PPO_Path = os.path.join("training", "saved models", "PPO_Model_CartPole")
model.save(PPO_Path)

D:\My_Projects\reinforcement-learning-lab\.venv\Lib\site-packages\stable_baselines3\common\save_util.py:284: UserWarning: Path 'training\saved models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [30]:
del model
model = PPO.load(PPO_Path, env=env)

# Evaluation